In [1]:
import bokeh.plotting
bokeh.plotting.output_notebook()

Loading BokehJS ...

In [5]:
import astropy.io.votable as votab

import numpy as np

latest_RV_url = "https://raw.githubusercontent.com/dsavransky/EXOSIMS/master/EXOSIMS/PlanetPopulation/RVplanets_ipac_2016-05-15.votable"
exoarch_confirmed_url = "http://exoplanetarchive.ipac.caltech.edu/cgi-bin/nstedAPI/nph-nstedAPI?table=exoplanets&format=votable"
exoplanets_eu = "http://www.exoplanet.eu/catalog/votable/" #times out
EXOSIM_stars = "https://raw.githubusercontent.com/dsavransky/EXOSIMS/master/EXOSIMS/StarCatalog/mission_exocat.votable"



exocat = votab.parse(EXOSIM_stars,).get_first_table().to_table()
#knownRV = votab.parse("../EXOSIMS/EXOSIMS/PlanetPopulation/RVplanets_ipac_032216.votable").get_first_table().to_table()
knownRV = votab.parse(latest_RV_url).get_first_table().to_table()
exoarchConfirmed = votab.parse(exoarch_confirmed_url).get_first_table().to_table()
#exoplanets_dot_eu = votab.parse(exoplanets_eu).get_first_table().to_table()


In [19]:

TOOLS="hover,crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"

p = bokeh.plotting.figure(tools=TOOLS)

p.scatter(exocat['st_teff'], (exocat['st_mass']), radius=50,
          fill_color='gray', fill_alpha=0.5,legend="ExoCat",
          line_color=None)

p.scatter(latestRV['st_teff'], (latestRV["st_mass"]), radius=100,
fill_color="orange", fill_alpha=0.2,legend="Known RV",
          line_color=None)
p.xaxis.axis_label = "T [K]"
p.yaxis.axis_label = "Mass [M_sun]"
bokeh.plotting.show(p)


In [6]:
p = bokeh.plotting.figure(tools=TOOLS, x_range=(0, 10000), y_range=(-5, 35) )


p.scatter(exocat['st_teff'], (exocat['st_dist']), radius=50,
          fill_color='gray', fill_alpha=0.2,legend="ExoCat",
          line_color=None)

p.scatter(knownRV['st_teff'], (knownRV["st_dist"]), radius=100,
fill_color="blue", fill_alpha=0.2,legend="Known RV",
          line_color=None)

p.scatter(exoarchConfirmed['st_teff'], (exoarchConfirmed["st_dist"]), radius=100,
fill_color="green", fill_alpha=0.2,legend="IPAC Exoplanet Archive",
          line_color=None)
p.xaxis.axis_label = "T [K]"
p.yaxis.axis_label = "distance [pc]"
bokeh.plotting.show(p)



In [7]:
p = bokeh.plotting.figure(tools=TOOLS, x_range=(0, 10000), y_range=(-5, 35) )


p.scatter(exocat['st_teff'], (exocat['st_dist']), radius=50,
          fill_color='gray', fill_alpha=0.2,legend="ExoCat",
          line_color=None)

p.scatter(knownRV['st_teff'], (knownRV["st_dist"]), radius=100,
fill_color="blue", fill_alpha=0.2,legend="Known RV",
          line_color=None)

p.scatter(exoarchConfirmed['st_teff'], (exoarchConfirmed["st_dist"]), radius=100,
fill_color="green", fill_alpha=0.2,legend="IPAC Exoplanet Archive",
          line_color=None)
p.xaxis.axis_label = "T [K]"
p.yaxis.axis_label = "distance [pc]"
bokeh.plotting.show(p)



In [8]:
def scatter_with_labels(p,x,y,names,color,legend,):
    source = bokeh.models.ColumnDataSource(data=dict(
        x=x,
        y=y,
        name=[j.decode('utf-8') for j in names],
    ))

    p.scatter(x,y, size=10,
          fill_color=color, fill_alpha=0.4,
              legend=legend,
          line_color=None,source=source)


In [9]:
hover = bokeh.models.HoverTool(
            tooltips=[
                ("index", "$index"),
                ("(x,y)", "($x, $y)"),
                ("name", "@name"),
            ]
        )
tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]
p = bokeh.plotting.figure(tools=tools,x_axis_type="log",  y_range=(-5, 35))#x_range=(0, 10000),  )

scatter_with_labels(p,exoarchConfirmed["pl_orbper"],exoarchConfirmed["st_dist"],
                    exoarchConfirmed["pl_hostname"],
                    color="blue",legend="IPAC Exoplanet Archive")

scatter_with_labels(p,knownRV["pl_orbper"],knownRV["st_dist"], 
                    knownRV["pl_hostname"],
                    color="green",legend="Known RV")
p.xaxis.axis_label = "orbital period [d]"
p.yaxis.axis_label = "distance [pc]"



bokeh.plotting.show(p)



In [18]:
hover = bokeh.models.HoverTool(
            tooltips=[
                ("index", "$index"),
                ("(x,y)", "($x, $y)"),
                ("name", "@name"),
            ]
        )


tools=[hover]+["crosshair,pan,wheel_zoom,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select"]

p = bokeh.plotting.figure(tools=tools,x_axis_type="log",  y_range=(-5, 35),x_range=(1e-3, 1000),  )

scatter_with_labels(p,exoarchConfirmed["pl_bmassj"],exoarchConfirmed["st_dist"],
                    exoarchConfirmed["pl_hostname"],
                    color="blue",legend="IPAC Exoplanet Archive")

scatter_with_labels(p,knownRV["pl_msinij"],knownRV["st_dist"], 
                    knownRV["pl_hostname"],
                    color="green",legend="Known RV")

p.xaxis.axis_label = "msini [Jupiter]"
p.yaxis.axis_label = "distance [pc]"



bokeh.plotting.show(p)

